In [1]:
#Libraries:
import datetime
import hashlib
import json
from flask import Flask, jsonify

In [2]:
#Blockchain 
class Blockchain:
    
    def __init__(self):
        self.chain = []
        self.create_block(proof = 1, previous_hash = '0')
    
    def create_block(self, proof, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash} #not for genesis block
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1] #index of the previous block

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**3 - previous_proof**2).encode()).hexdigest() #mathematical problem to create suitable hash
            if hash_operation[:4] == '0000': #standard difficulty, 
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**3 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True

In [3]:
#Mine

#Flask webapp
#Call class
app = Flask(__name__)


blockchain = Blockchain()

#To mine
@app.route('/mine_block', methods = ['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    response = {'message': 'A Block Has Been Mined',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash']}
    return jsonify(response), 200

#View blockchain in app
@app.route('/get_chain', methods = ['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

#Validity
@app.route('/is_valid', methods = ['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'Valididty Confirmed'}
    else:
        response = {'message': 'Validity Denied'}
    return jsonify(response), 200

In [4]:
#Running:
#Install latest flask
#Run in IDE/CMD
#Go to postman, select GET, paste http://127.0.0.1:5000/ and select one of the routes
app.run(host = '0.0.0.0', port = 5000)(debug=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.204:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2021 00:28:59] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:24] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:27] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:31] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:33] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:36] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:37] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:29:48] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 00:30:33] "GET /is_valid HTTP/1.1" 200 -


TypeError: 'NoneType' object is not callable